# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 10 2023 10:58AM,257134,16,EMERSA-9140918,"Emersa Waterbox, LLC",Released
1,Feb 10 2023 10:55AM,257133,10,Enova-11342,Emerginnova,Released
2,Feb 10 2023 10:54AM,257132,10,37600,"Digital Brands, LLC",Released
3,Feb 10 2023 10:54AM,257132,10,37601,"Digital Brands, LLC",Released
4,Feb 10 2023 10:50AM,257131,10,9140912,Emerginnova,Released
5,Feb 10 2023 10:30AM,257127,10,PRF-52464,Bio-PRF,Released
6,Feb 10 2023 10:30AM,257127,10,PRF-52469,Bio-PRF,Released
7,Feb 10 2023 10:30AM,257127,10,PRF-52475,Bio-PRF,Released
8,Feb 10 2023 10:30AM,257127,10,PRF-52478,Bio-PRF,Released
9,Feb 10 2023 10:30AM,257127,10,PRF-52482,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,257127,Released,7
39,257131,Released,1
40,257132,Released,2
41,257133,Released,1
42,257134,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257127,NaN,7.0
257131,NaN,1.0
257132,NaN,2.0
257133,NaN,1.0
257134,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256949,32.0,15.0
257005,0.0,3.0
257047,0.0,3.0
257053,0.0,2.0
257055,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256949,32,15
257005,0,3
257047,0,3
257053,0,2
257055,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256949,32,15
1,257005,0,3
2,257047,0,3
3,257053,0,2
4,257055,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256949,32,15
1,257005,,3
2,257047,,3
3,257053,,2
4,257055,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 10 2023 10:58AM,257134,16,"Emersa Waterbox, LLC"
1,Feb 10 2023 10:55AM,257133,10,Emerginnova
2,Feb 10 2023 10:54AM,257132,10,"Digital Brands, LLC"
4,Feb 10 2023 10:50AM,257131,10,Emerginnova
5,Feb 10 2023 10:30AM,257127,10,Bio-PRF
12,Feb 10 2023 10:21AM,257126,16,Sartorius Stedim Filters Inc.
13,Feb 10 2023 10:15AM,257125,10,ISDIN Corporation
16,Feb 10 2023 9:51AM,257124,10,Eye Pharma Inc
17,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.
18,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 10 2023 10:58AM,257134,16,"Emersa Waterbox, LLC",,1
1,Feb 10 2023 10:55AM,257133,10,Emerginnova,,1
2,Feb 10 2023 10:54AM,257132,10,"Digital Brands, LLC",,2
3,Feb 10 2023 10:50AM,257131,10,Emerginnova,,1
4,Feb 10 2023 10:30AM,257127,10,Bio-PRF,,7
5,Feb 10 2023 10:21AM,257126,16,Sartorius Stedim Filters Inc.,,1
6,Feb 10 2023 10:15AM,257125,10,ISDIN Corporation,,3
7,Feb 10 2023 9:51AM,257124,10,Eye Pharma Inc,,1
8,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.,,1
9,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 10:58AM,257134,16,"Emersa Waterbox, LLC",1,
1,Feb 10 2023 10:55AM,257133,10,Emerginnova,1,
2,Feb 10 2023 10:54AM,257132,10,"Digital Brands, LLC",2,
3,Feb 10 2023 10:50AM,257131,10,Emerginnova,1,
4,Feb 10 2023 10:30AM,257127,10,Bio-PRF,7,
5,Feb 10 2023 10:21AM,257126,16,Sartorius Stedim Filters Inc.,1,
6,Feb 10 2023 10:15AM,257125,10,ISDIN Corporation,3,
7,Feb 10 2023 9:51AM,257124,10,Eye Pharma Inc,1,
8,Feb 10 2023 9:38AM,257123,16,Sartorius Stedim Filters Inc.,1,
9,Feb 10 2023 9:34AM,257066,15,"Virtus Pharmaceuticals, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 10:58AM,257134,16,"Emersa Waterbox, LLC",1,
1,Feb 10 2023 10:55AM,257133,10,Emerginnova,1,
2,Feb 10 2023 10:54AM,257132,10,"Digital Brands, LLC",2,
3,Feb 10 2023 10:50AM,257131,10,Emerginnova,1,
4,Feb 10 2023 10:30AM,257127,10,Bio-PRF,7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 10:58AM,257134,16,"Emersa Waterbox, LLC",1.0,NaN
1,Feb 10 2023 10:55AM,257133,10,Emerginnova,1.0,NaN
2,Feb 10 2023 10:54AM,257132,10,"Digital Brands, LLC",2.0,NaN
3,Feb 10 2023 10:50AM,257131,10,Emerginnova,1.0,NaN
4,Feb 10 2023 10:30AM,257127,10,Bio-PRF,7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 10 2023 10:58AM,257134,16,"Emersa Waterbox, LLC",1.0,0.0
1,Feb 10 2023 10:55AM,257133,10,Emerginnova,1.0,0.0
2,Feb 10 2023 10:54AM,257132,10,"Digital Brands, LLC",2.0,0.0
3,Feb 10 2023 10:50AM,257131,10,Emerginnova,1.0,0.0
4,Feb 10 2023 10:30AM,257127,10,Bio-PRF,7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5656234,126.0,1.0
12,514137,2.0,0.0
15,1542241,26.0,40.0
16,1285611,5.0,0.0
18,257106,1.0,0.0
19,771305,15.0,4.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5656234,126.0,1.0
1,12,514137,2.0,0.0
2,15,1542241,26.0,40.0
3,16,1285611,5.0,0.0
4,18,257106,1.0,0.0
5,19,771305,15.0,4.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,126.0,1.0
1,12,2.0,0.0
2,15,26.0,40.0
3,16,5.0,0.0
4,18,1.0,0.0
5,19,15.0,4.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,126.0
1,12,Released,2.0
2,15,Released,26.0
3,16,Released,5.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19
Status,,,,,,
Executing,1.0,0.0,40.0,0.0,0.0,4.0
Released,126.0,2.0,26.0,5.0,1.0,15.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19
0,Executing,1.0,0.0,40.0,0.0,0.0,4.0
1,Released,126.0,2.0,26.0,5.0,1.0,15.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19
0,Executing,1.0,0.0,40.0,0.0,0.0,4.0
1,Released,126.0,2.0,26.0,5.0,1.0,15.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()